In [2]:
import math as mt 
import numpy as np 
import math
from numpy.lib.stride_tricks import sliding_window_view
import torch.nn.functional as F
import torch
from numpy_cnn_operations import *
device="cuda" if  torch.cuda.is_available() else "cpu"
torch.cuda.device_count()
device


'cpu'

numpy

torch

In [3]:


def padding_torch(img_array_perm,padding,stride,kernel_size):
        if padding: 

            #this padding is correct as can i allow assymetric padding and i use ALL pixels at borders
            H=img_array_perm[0,0,:,:].shape[0]
            W=img_array_perm[0,0,:,:].shape[1]

            p_top, p_bottom, p_left, p_right=calculate_optimal_padding(H,W,f=kernel_size,s=stride)

            img_array_perm=F.pad(img_array_perm,(p_left,p_right,p_top,p_bottom))
        return img_array_perm

def convolution_torch(img_array_perm,kernels_perm,bias,padding=True,stride=1):
    kernels_perm=torch.flip(kernels_perm, dims=[2, 3])
    kernel_size=kernels_perm.shape[2]
    img_array_perm=padding_torch(img_array_perm,padding,stride,kernel_size)
    out = F.conv2d(
        img_array_perm,          # shape (N, C_in, H, W)
        kernels_perm,    # shape (C_out, C_in, kH, kW)
        stride=stride,   
    )
    return out.add_(bias.reshape(1,len(kernels_perm),1,1))

def maxpooling_torch(img_array_perm,kernel_size,stride=2,padding=True):

    img_array_perm=padding_torch(img_array_perm,padding,stride,kernel_size)

    out = F.max_pool2d(
        img_array_perm,
        kernel_size=kernel_size,  # size of pooling window
        stride=stride,     
    )
    return out

torch

In [5]:
nb_images=10
nb_filters=7
if device=="cuda":
    torch.cuda.manual_seed(123)
else:
    torch.manual_seed(123)
img_array = torch.randn(nb_images,3,25, 25,device=device)

kernels1 = torch.randn(nb_filters,3,2, 2,device=device)
bias1=torch.arange(nb_filters,device=device)


conv1=convolution_torch(img_array,kernels1,bias1,padding=True,stride=1)
mx=maxpooling_torch(conv1,kernel_size=2,stride=2,padding=True)

nb_filters=nb_filters*2


kernels2 = torch.randn(nb_filters,int(nb_filters/2),2, 2,device=device)
bias2=torch.arange(nb_filters,device=device)

conv1=convolution_torch(mx,kernels2,bias2,padding=True,stride=1)
mx=maxpooling_torch(conv1,kernel_size=2,stride=2,padding=True)

nb_filters=nb_filters*2


kernels3 = torch.randn(nb_filters,int(nb_filters/2),2, 2,device=device)
bias3=torch.arange(nb_filters,device=device)

conv1=convolution_torch(mx,kernels3,bias3,padding=True,stride=1)
mx=maxpooling_torch(conv1,kernel_size=2,stride=2,padding=True)
mx[0,0,:,:]



tensor([[  89.5955,  171.0315,  169.7924,  174.2054],
        [  45.9423,   19.2987,   57.5251,  -18.9793],
        [   0.0000, -102.0316,  -56.7250,  -66.5183],
        [   0.0000, -142.2567,  -45.2281,  -94.5289]])

In [6]:
mx.shape

torch.Size([10, 28, 4, 4])

In [9]:
mx.shape[0]

10

# multi  image CNN Forward pass

numpy

In [134]:


actual=img_array.permute(2,3,1,0).numpy()
kernels3=kernels3.permute(2,3,1,0).numpy()
kernels2=kernels2.permute(2,3,1,0).numpy()
kernels1=kernels1.permute(2,3,1,0).numpy()
bias3=bias3.numpy()
bias2=bias2.numpy()
bias1=bias1.numpy()

nb_filters = 7



conv1=conv3D(x=actual,nb_filters=nb_filters,kernel=kernels1 ,bias=bias1,padding=True)
print("conv1 ",conv1.shape)

mp1=MaxPooling3D(conv1,filter_f=2,stride=2)
print("mp1 ",mp1.shape)

actual=mp1



nb_filters = nb_filters*2



conv1=conv3D(x=actual,nb_filters=nb_filters,kernel=kernels2 ,bias=bias2,padding=True)
print("conv1 ",conv1.shape)

mp1=MaxPooling3D(conv1,filter_f=2,stride=2)
print("mp1 ",mp1.shape)


actual=mp1



nb_filters = nb_filters*2



conv1=conv3D(x=actual,nb_filters=nb_filters,kernel=kernels3 ,bias=bias3,padding=True)
print("conv1 ",conv1.shape)

mp1=MaxPooling3D(conv1,filter_f=2,stride=2)
print("mp1 ",mp1.shape)



1 0 1 0
conv1  (25, 25, 7, 1000)
1 0 1 0
mp1  (13, 13, 7, 1000)
1 0 1 0
conv1  (13, 13, 14, 1000)
1 0 1 0
mp1  (7, 7, 14, 1000)
1 0 1 0
conv1  (7, 7, 28, 1000)
1 0 1 0
mp1  (4, 4, 28, 1000)


In [123]:
mp1[:,:,0,0]

array([[   0.        ,   11.39261206,   24.92602323,   54.52905878],
       [   0.        , -115.0364984 , -197.72659355, -262.25516889],
       [  30.91247531,  -60.46000182, -153.98798617, -268.18114204],
       [   0.        , -115.68519437, -190.76068574, -232.33123386]])

In [125]:
mx[0,0,:,:]

tensor([[   0.0000,   11.3926,   24.9260,   54.5291],
        [   0.0000, -115.0365, -197.7266, -262.2551],
        [  30.9125,  -60.4600, -153.9879, -268.1811],
        [   0.0000, -115.6852, -190.7607, -232.3313]])

torch

# one  image CNN Forward pass

In [125]:
np.random.seed(123)
nb_images=15
img_array = np.random.randn(100, 100, 3)

In [126]:
print("img ",img_array.shape)

actual=img_array

nb_filters = 7
kernels = np.random.randn(2, 2, 3,nb_filters)
bias=np.arange(nb_filters)


conv1=conv3D(x=actual,nb_filters=nb_filters,kernel=kernels ,bias=bias,padding=True)
print("conv1 ",conv1.shape)

mp1=MaxPooling3D(conv1,filter_f=2,stride=2)
print("mp1 ",mp1.shape)

actual=mp1



nb_filters = nb_filters*2
kernels = np.random.randn(2, 2, int(nb_filters/2),nb_filters)
bias=np.arange(nb_filters)


conv1=conv3D(x=actual,nb_filters=nb_filters,kernel=kernels ,bias=bias,padding=True)
print("conv1 ",conv1.shape)

mp1=MaxPooling3D(conv1,filter_f=2,stride=2)
print("mp1 ",mp1.shape)


actual=mp1



nb_filters = nb_filters*2
kernels = np.random.randn(2, 2, int(nb_filters/2),nb_filters)
bias=np.arange(nb_filters)


conv1=conv3D(x=actual,nb_filters=nb_filters,kernel=kernels ,bias=bias,padding=True)
print("conv1 ",conv1.shape)

mp1=MaxPooling3D(conv1,filter_f=2,stride=2)
print("mp1 ",mp1.shape)



img  (100, 100, 3)
1 0 1 0
conv1  (100, 100, 7)
0 0 0 0
mp1  (50, 50, 7)
1 0 1 0
conv1  (50, 50, 14)
0 0 0 0
mp1  (25, 25, 14)
1 0 1 0
conv1  (25, 25, 28)
1 0 1 0
mp1  (13, 13, 28)
